### Merge Data

#### Merge Pediatrics data with Drug and Reaction
- Pediatrics data from 3_Pediatrics_data_merge.ipynb file
- Raw Drug file from drugcharacteristics.csv.gz (Drug)
- Reaction file from reactions.csv.gz (Meddra reaction PT)

In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

#### Raw Data

In [50]:
ped_report_data = pd.read_csv('../../data/pediatric_patients_report_serious_reporter.csv.gz',compression='gzip', index_col=0)

/tmp/ipykernel_3670/852322303.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  ped_report_data = pd.read_csv('../../data/pediatric_patients_report_serious_reporter.csv.gz',compression='gzip', index_col=0)


In [51]:
drug_characteristics_df = pd.read_csv('../../data/openFDA_drug_event/er_tables_memory_efficient/drugcharacteristics.csv.gz',compression='gzip')

/tmp/ipykernel_3670/195607322.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  drug_characteristics_df = pd.read_csv('../../data/openFDA_drug_event/er_tables_memory_efficient/drugcharacteristics.csv.gz',compression='gzip')


In [52]:
drug_characteristics_df

,drug_administration,drug_characterization,drug_indication,medicinal_product,safetyreportid
0,Intravenous (not otherwise specified),Suspect (the drug was considered by the report...,OSTEOPOROSIS,BONIVA,10003300
1,NaN,Suspect (the drug was considered by the report...,PRODUCT USED FOR UNKNOWN INDICATION,IBUPROFEN,10003301
2,NaN,Suspect (the drug was considered by the report...,NaN,LYRICA,10003302
3,NaN,Suspect (the drug was considered by the report...,NaN,FIORINAL,10003304
4,NaN,Suspect (the drug was considered by the report...,NaN,DOXYCYCLINE HYCLATE,10003304
...,...,...,...,...,...
64177915,Oral,Suspect (the drug was considered by the report...,PRODUCT USED FOR UNKNOWN INDICATION,LETAIRIS,9999996
64177916,NaN,Suspect (the drug was considered by the report...,NaN,VENTAVIS,9999997
64177917,NaN,Suspect (the drug was considered by the report...,COR PULMONALE CHRONIC,LETAIRIS,9999997
64177918,Oral,Suspect (the drug was considered by the report...,PRODUCT USED FOR UNKNOWN INDICATION,LETAIRIS,9999998


In [53]:
reaction_df = pd.read_csv("../../data/openFDA_drug_event/er_tables_memory_efficient/reactions.csv.gz", compression='gzip', low_memory=False)

In [54]:
reaction_df

,reaction_meddrapt,reaction_outcome,safetyreportid
0,Diarrhoea,NaN,10003300
1,Arthralgia,NaN,10003300
2,Headache,NaN,10003300
3,Vomiting,NaN,10003300
4,Dyspepsia,NaN,10003301
...,...,...,...
56821719,Dizziness,Unknown,9999997
56821720,Cough,Unknown,9999997
56821721,Oedema,Unknown,9999998
56821722,Chest pain,Unknown,9999999


##### Merge Data

In [55]:
ped_data = ped_report_data.copy()

In [56]:
# ped_report_data.isna().sum()
ped_report_data.shape

(859343, 21)

In [57]:
ped_data_merge = ped_report_data\
    .merge(drug_characteristics_df, on='safetyreportid', how='inner')\
    .merge(reaction_df, on='safetyreportid', how='inner')

##### Pediatrics Data Merge

In [58]:
ped_data_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754671 entries, 0 to 1754670
Data columns (total 27 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   patient_custom_master_age  float64
 1   patient_onsetage           float64
 2   patient_onsetageunit       object 
 3   patient_sex                object 
 4   patient_weight             float64
 5   safetyreportid             object 
 6   nichd                      object 
 7   ich_ema                    object 
 8   fda                        object 
 9   lastupdate_date            int64  
 10  mostrecent_receive_date    int64  
 11  receive_date               int64  
 12  congenital_anomali         float64
 13  death                      float64
 14  disabling                  float64
 15  life_threatening           float64
 16  other                      float64
 17  serious                    object 
 18  reporter_company           object 
 19  reporter_country           object 
 20  re

##### Missing Data

In [59]:
missing_data_merge = ped_data_merge.isna().sum().sort_values(ascending=False)
missing_data_merge

congenital_anomali           1718001
disabling                    1654300
reaction_outcome             1653173
life_threatening             1537713
death                        1429485
drug_administration           960949
patient_weight                906664
drug_indication               873247
other                         866181
fda                           699833
ich_ema                       479374
reporter_country              292501
reporter_qualification        288592
reporter_company               99597
patient_sex                    37526
medicinal_product                 34
patient_onsetage                   0
patient_onsetageunit               0
patient_custom_master_age          0
mostrecent_receive_date            0
lastupdate_date                    0
safetyreportid                     0
nichd                              0
serious                            0
receive_date                       0
drug_characterization              0
reaction_meddrapt                  0
d

#### Age Overview

In [60]:
# check  age unit
ped_data_merge['patient_onsetageunit'].value_counts(ascending=False)

patient_onsetageunit
Year      1498240
Month      157629
Day         78514
Week        13428
Hour         5810
Decade       1050
Name: count, dtype: int64

In [61]:
#Recheck Age after filter
ped_data_merge.loc[(ped_data_merge['patient_onsetageunit'] == 'Month'),['patient_custom_master_age','patient_onsetage','patient_onsetageunit']].sort_values(by='patient_onsetage',ascending=False)

,patient_custom_master_age,patient_onsetage,patient_onsetageunit
1224610,20.916667,251.0,Month
1162528,20.916667,251.0,Month
1162529,20.916667,251.0,Month
1162526,20.916667,251.0,Month
1162527,20.916667,251.0,Month
...,...,...,...
723418,0.083333,1.0,Month
723419,0.083333,1.0,Month
723420,0.083333,1.0,Month
723421,0.083333,1.0,Month


In [62]:
# print(type(ped_data_merge))

In [63]:
# print(ped_data_merge)

In [64]:
# Add new column with 'Year' as the value for all rows
ped_data_merge['patient_custom_master_age_unit'] = 'Year'

# Reorder columns to place the new column after 'patient_custom_master_age'
cols = list(ped_data_merge.columns)
cols.insert(cols.index('patient_custom_master_age') + 1, cols.pop(cols.index('patient_custom_master_age_unit')))
ped_data_merge = ped_data_merge[cols]

In [65]:
ped_data_merge.isna().sum().sort_values(ascending=False)

congenital_anomali                1718001
disabling                         1654300
reaction_outcome                  1653173
life_threatening                  1537713
death                             1429485
drug_administration                960949
patient_weight                     906664
drug_indication                    873247
other                              866181
fda                                699833
ich_ema                            479374
reporter_country                   292501
reporter_qualification             288592
reporter_company                    99597
patient_sex                         37526
medicinal_product                      34
patient_custom_master_age               0
patient_custom_master_age_unit          0
patient_onsetage                        0
patient_onsetageunit                    0
mostrecent_receive_date                 0
lastupdate_date                         0
safetyreportid                          0
nichd                             

In [66]:
ped_data_merge.shape

(1754671, 28)

### Data Cleaning

#### Fill NA and Unknow

In [67]:
#fill NA in columns congenital_anomali,disabling,life_threatening,death,other
cols = ['congenital_anomali','disabling','life_threatening','death','other']
for i in cols:
    print(ped_data_merge[i].unique())

existing_cols = [col for col in cols if col in ped_data_merge.columns]

for col in existing_cols:
    ped_data_merge[col] = ped_data_merge[col].fillna(0).astype(int)
    
# After filling na
print('')
print('After filling NA : \n')
for i in cols:
    print(ped_data_merge[i].unique())

[ 2.  1. nan]
[ 2.  1. nan]
[ 2.  1. nan]
[ 2.  1. nan]
[ 2.  1. nan]

After filling NA : 

[2 1 0]
[2 1 0]
[2 1 0]
[2 1 0]
[2 1 0]


In [68]:
# fill unknown value in columns about patient
categorical_cols_to_fill_unknown = ['patient_sex']
for col in categorical_cols_to_fill_unknown:
    ped_data_merge[col] = ped_data_merge[col].fillna('Unknown')

In [69]:
# fill unknown value in columns about reporter
categorical_cols_to_fill_unknown = ['reporter_company', 'reporter_qualification', 'reporter_country']
for col in categorical_cols_to_fill_unknown:
    ped_data_merge[col] = ped_data_merge[col].fillna('Unknown')

In [70]:
# fill unknown value in columns about reaction
categorical_cols_to_fill_unknown = ['reaction_outcome']
for col in categorical_cols_to_fill_unknown:
    ped_data_merge[col] = ped_data_merge[col].fillna('Unknown')

In [71]:
# fill unknown value in columns drug_administration
categorical_cols_to_fill_unknown = ['drug_administration']
for col in categorical_cols_to_fill_unknown:
    ped_data_merge[col] = ped_data_merge[col].fillna('Unknown')

In [72]:
ped_data_merge.isna().sum().sort_values(ascending=False)

patient_weight                    906664
drug_indication                   873247
fda                               699833
ich_ema                           479374
medicinal_product                     34
patient_onsetage                       0
patient_onsetageunit                   0
patient_sex                            0
nichd                                  0
safetyreportid                         0
patient_custom_master_age_unit         0
patient_custom_master_age              0
mostrecent_receive_date                0
lastupdate_date                        0
receive_date                           0
congenital_anomali                     0
life_threatening                       0
other                                  0
death                                  0
disabling                              0
reporter_company                       0
serious                                0
reporter_country                       0
reporter_qualification                 0
drug_characteriz

In [73]:
ped_data_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754671 entries, 0 to 1754670
Data columns (total 28 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   patient_custom_master_age       float64
 1   patient_custom_master_age_unit  object 
 2   patient_onsetage                float64
 3   patient_onsetageunit            object 
 4   patient_sex                     object 
 5   patient_weight                  float64
 6   safetyreportid                  object 
 7   nichd                           object 
 8   ich_ema                         object 
 9   fda                             object 
 10  lastupdate_date                 int64  
 11  mostrecent_receive_date         int64  
 12  receive_date                    int64  
 13  congenital_anomali              int64  
 14  death                           int64  
 15  disabling                       int64  
 16  life_threatening                int64  
 17  other                      

#### Datetime format

In [74]:
print(ped_data_merge[['lastupdate_date', 'mostrecent_receive_date', 'receive_date']].head())
date_cols_to_convert = ['lastupdate_date', 'mostrecent_receive_date', 'receive_date']

for col in date_cols_to_convert:
    ped_data_merge[col] = pd.to_datetime(ped_data_merge[col].astype(str), format='%Y%m%d', errors='coerce')

   lastupdate_date  mostrecent_receive_date  receive_date
0         20250409                 20250327      20250327
1         20250409                 20250327      20250327
2         20250409                 20250327      20250327
3         20250409                 20250327      20250327
4         20250409                 20250327      20250327


In [75]:
print(ped_data_merge[['lastupdate_date', 'mostrecent_receive_date', 'receive_date']].head())
print(ped_data_merge.info())

  lastupdate_date mostrecent_receive_date receive_date
0      2025-04-09              2025-03-27   2025-03-27
1      2025-04-09              2025-03-27   2025-03-27
2      2025-04-09              2025-03-27   2025-03-27
3      2025-04-09              2025-03-27   2025-03-27
4      2025-04-09              2025-03-27   2025-03-27
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754671 entries, 0 to 1754670
Data columns (total 28 columns):
 #   Column                          Dtype         
---  ------                          -----         
 0   patient_custom_master_age       float64       
 1   patient_custom_master_age_unit  object        
 2   patient_onsetage                float64       
 3   patient_onsetageunit            object        
 4   patient_sex                     object        
 5   patient_weight                  float64       
 6   safetyreportid                  object        
 7   nichd                           object        
 8   ich_ema                         

In [76]:
ped_data_merge.shape

(1754671, 28)

#### Droup Data

In [77]:
# Droup medicinal_product because have missing 34 rows
ped_data_merge = ped_data_merge.dropna(subset=['medicinal_product'])

In [78]:
# ลบคอลัมน์ 'patient_weight'
ped_data_merge = ped_data_merge.drop(columns=['patient_weight'])

In [79]:
ped_data_merge =ped_data_merge.dropna(subset=['drug_indication'])

In [80]:
ped_data_merge =ped_data_merge.dropna(subset=['mostrecent_receive_date'])

In [81]:
ped_data_merge.isna().sum().sort_values(ascending=False)

fda                               351537
ich_ema                           241452
patient_custom_master_age              0
patient_onsetage                       0
patient_custom_master_age_unit         0
patient_onsetageunit                   0
patient_sex                            0
nichd                                  0
safetyreportid                         0
lastupdate_date                        0
mostrecent_receive_date                0
receive_date                           0
congenital_anomali                     0
death                                  0
disabling                              0
life_threatening                       0
other                                  0
serious                                0
reporter_company                       0
reporter_country                       0
reporter_qualification                 0
drug_administration                    0
drug_characterization                  0
drug_indication                        0
medicinal_produc

In [82]:
ped_data_merge.shape

(881396, 27)

#### Polyphamacy

In [ ]:
# unique_drugs_per_report = ped_data_merge.groupby('safetyreportid')['medicinal_product'].nunique().reset_index()
# unique_drugs_per_report.columns = ['safetyreportid', 'unique_drug_count']

#### Save Files

In [83]:
ped_data_merge.to_csv('../../data/pediatric_patients_report_drug_reaction.csv.gz',compression='gzip')

In [84]:
data = pd.read_csv(
    '../../data/pediatric_patients_report_drug_reaction.csv.gz',
    compression='gzip',
    low_memory=False,
    index_col=0
)

sampled_data = data.sample(n=1000, random_state=42)

print("ข้อมูลสุ่ม 1,000 แถว:")
print("\nจำนวนแถวและคอลัมน์:", sampled_data.shape)
print("\nคอลัมน์ทั้งหมด:", sampled_data.columns.tolist())
display(sampled_data.head(10))


sampled_data.to_csv('../../data/sampled_pediatric_data.csv', index=False)
sampled_data.to_excel('../../data/sampled_pediatric_data.xlsx', index=False)

ข้อมูลสุ่ม 1,000 แถว:

จำนวนแถวและคอลัมน์: (1000, 27)

คอลัมน์ทั้งหมด: ['patient_custom_master_age', 'patient_custom_master_age_unit', 'patient_onsetage', 'patient_onsetageunit', 'patient_sex', 'safetyreportid', 'nichd', 'ich_ema', 'fda', 'lastupdate_date', 'mostrecent_receive_date', 'receive_date', 'congenital_anomali', 'death', 'disabling', 'life_threatening', 'other', 'serious', 'reporter_company', 'reporter_country', 'reporter_qualification', 'drug_administration', 'drug_characterization', 'drug_indication', 'medicinal_product', 'reaction_meddrapt', 'reaction_outcome']


,patient_custom_master_age,patient_custom_master_age_unit,patient_onsetage,patient_onsetageunit,patient_sex,safetyreportid,nichd,ich_ema,fda,lastupdate_date,...,serious,reporter_company,reporter_country,reporter_qualification,drug_administration,drug_characterization,drug_indication,medicinal_product,reaction_meddrapt,reaction_outcome
92034,0.041096,Year,15.0,Day,Male,4381566-7,term_neonatal,term_newborn_infants,neonates,2005-01-07,...,"The adverse event resulted in death, a life th...",HK-MERCK-0406HKG00006,Unknown,Physician,Intravenous (not otherwise specified),Suspect (the drug was considered by the report...,PATENT DUCTUS ARTERIOSUS,INDOCIN I.V.,THROMBOCYTOPENIA,Unknown
760022,8.000000,Year,8.0,Year,Male,5822566-7,middle_childhood,children,children,2009-01-09,...,"The adverse event resulted in death, a life th...",Unknown,UNITED STATES,Physician,Intravenous (not otherwise specified),Suspect (the drug was considered by the report...,BRAIN NEOPLASM,CYCLOPHOSPHAMIDE,BODY TEMPERATURE DECREASED,Unknown
872536,1.583333,Year,19.0,Month,Female,6125174,toddler,infants_and_toddlers,infants,2023-04-18,...,"The adverse event resulted in death, a life th...",FR-SANOFI-SYNTHELABO-A02200602217,FR,Other health professional,Intravenous (not otherwise specified),Suspect (the drug was considered by the report...,Acute myeloid leukaemia,DAUNORUBICIN,Cardiac failure,Fatal
223633,20.000000,Year,20.0,Year,Male,4620495-8,late_adolescence,NaN,NaN,2005-07-27,...,"The adverse event resulted in death, a life th...",Unknown,Unknown,Consumer or non-health professional,Unknown,Suspect (the drug was considered by the report...,ACNE CYSTIC,ACCUTANE,DRUG EFFECT DECREASED,Unknown
1306197,6.000000,Year,6.0,Year,Female,7443126-0,middle_childhood,children,children,2011-10-10,...,"The adverse event resulted in death, a life th...",A0745211A,UNITED STATES,Consumer or non-health professional,Respiratory (inhalation),Concomitant (the drug was reported as being ta...,ASTHMA,ALBUTEROL,DYSPNOEA EXERTIONAL,Unknown
233464,14.000000,Year,14.0,Year,Male,4637371-7,early_adolescence,adolescents,adolescents,2005-10-28,...,The adverse event did not result in any of the...,USA040977261,Unknown,Consumer or non-health professional,Unknown,Suspect (the drug was considered by the report...,ATTENTION DEFICIT/HYPERACTIVITY DISORDER,STRATTERA,HEADACHE,Unknown
1247316,0.416667,Year,5.0,Month,Male,7274900-1,infancy,infants_and_toddlers,infants,2011-08-31,...,"The adverse event resulted in death, a life th...",Unknown,UNITED STATES,Unknown,Dental,Suspect (the drug was considered by the report...,TEETHING,HYLANS TEETHING TABLETS TEETHING TABLETS,PYREXIA,Unknown
164225,21.000000,Year,21.0,Year,Female,4506283-1,late_adolescence,NaN,NaN,2005-03-28,...,"The adverse event resulted in death, a life th...",US-JNJFOC-20040101137,Unknown,Other health professional,Intravenous (not otherwise specified),Suspect (the drug was considered by the report...,CROHN'S DISEASE,REMICADE,ANAPHYLACTIC REACTION,Unknown
760385,6.000000,Year,6.0,Year,Male,5823190-2,middle_childhood,children,children,2009-01-09,...,"The adverse event resulted in death, a life th...",PHBS2008IT05182,ITALY,Other health professional,Oral,Suspect (the drug was considered by the report...,ACUTE LEUKAEMIA,CYTARABINE,SOMNOLENCE,Unknown
1752725,16.000000,Year,16.0,Year,Female,8640742,early_adolescence,adolescents,NaN,2013-06-27,...,"The adverse event resulted in death, a life th...",US-JNJFOC-20120610126,US,Physician,Intravenous (not otherwise specified),Suspect (the drug was considered by the report...,CROHN^S DISEASE,REMICADE,Failure to thrive,Recovered/resolved
